In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense
import csv
import math
from scipy import signal
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, concatenate
import keras.backend as K
import scipy
from tensorflow.keras import layers, models
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from tensorflow.keras.initializers import RandomNormal
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
import warnings
warnings.filterwarnings('ignore')
import pickle as pkl
import time
import os
from MultiScaleESN import MultiScaleESNLayer # import the multiscale esn framework from MultiScaleESN.py python file

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

print(tf.__version__)

print(tf.config.list_physical_devices('GPU'))

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.10.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Found GPU at: /device:GPU:0


In [12]:
# Replace the path with your own CSV file paths. Printing waveform shapes helps verify their dimensions.
def tab_get(file):
    df=pd.read_csv(file,sep=',')
    labels = []
    tabular_features = []
    for _, row in df.iterrows():
        selected_features=["PI_v","T_va","Tau_c","RSSC Vel","ID_2",	"IV_2","CAV"]
        tabular_features.append(row[selected_features])
        labels.append(row['Magnitude'])
    labels = np.array(labels)
    return tabular_features,labels

def create_image_dataset(time=600):
    test_file="E:/Reservoir Learning/Input Dataset/Test_tab_MAG_data_6_SEC.csv"
    tab_test,y_test =tab_get(test_file)
    print("tab_test",len(tab_test))
    test_file="E:/Reservoir Learning/Input Dataset/Train_tab_MAG_data_6_SEC.csv"
    tab_train,y_train =tab_get(test_file)
    print("tab_train",len(tab_train))
    test_file="E:/Reservoir Learning/Input Dataset/Val_tab_MAG_data_6_SEC.csv"
    tab_val,y_val =tab_get(test_file)
    print("tab_val",len(tab_val))
    # Load seismic waveform data
    file_path = os.path.expanduser("E:/Reservoir Learning/Input Dataset/Train_waveform_MAG_data_3C.npy")
    seismic_waveforms_acc = np.load(file_path)
    seismic_train_IN = seismic_waveforms_acc[:, :time]
    print("seismic_train_IN",seismic_train_IN.shape)
    file_path = os.path.expanduser("E:/Reservoir Learning/Input Dataset/Test_waveform_MAG_data_3C.npy")
    seismic_waveforms_vel = np.load(file_path)
    seismic_test_IN = seismic_waveforms_vel[:, :time]
    print("seismic_test_IN",seismic_test_IN.shape)
    file_path = os.path.expanduser("E:/Reservoir Learning/Input Dataset/Val_waveform_MAG_data_3C.npy")
    seismic_waveforms_disp = np.load(file_path)
    seismic_val_IN = seismic_waveforms_disp[:, :time]
    print("seismic_val_IN",seismic_val_IN.shape)
    return y_train,y_test,y_val, seismic_train_IN, seismic_test_IN, seismic_val_IN,tab_train,tab_test, tab_val
y_train,y_test,y_val, seismic_train, seismic_test, seismic_val,tab_train,tab_test, tab_val=create_image_dataset()
def change_to_tensor(arr):
    x_tensor = tf.convert_to_tensor(arr.astype(np.float32))
    return x_tensor
y_train=change_to_tensor(y_train)
y_test=change_to_tensor(y_test)
y_val=change_to_tensor(y_val)
tab_test=change_to_tensor(np.array(tab_test))
tab_train=change_to_tensor(np.array(tab_train))
tab_val=change_to_tensor(np.array(tab_val))

tab_test 10112
tab_train 23527
tab_val 3801
seismic_train_IN (23527, 600, 9)
seismic_test_IN (10112, 600, 9)
seismic_val_IN (3801, 600, 9)


In [4]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

In [10]:
def build_deepesn_model(ts_steps, output_dim=1):
    ts_input = tf.keras.Input(shape=(seismic_train.shape[1], seismic_train.shape[2]), name='timeseries')
    tabular_input = tf.keras.Input(shape=(7,), name='tabular')
    # Deep ESN 3 layers with multi rates
    x = MultiScaleESNLayer(300, leak_rates=[0.8, 1.0, 0.5, 0.1, 0.08],spectral_radius=1.2)(ts_input)
    print(x.shape)
    x = tf.expand_dims(x, axis=1)
    x = MultiScaleESNLayer(300, leak_rates=[0.8, 1.0, 0.5, 0.1, 0.08],spectral_radius=1.0)(x)
    print(x.shape)
    x = tf.expand_dims(x, axis=1)
    x = MultiScaleESNLayer(300, leak_rates=[0.8, 1.0, 0.5, 0.1, 0.08],spectral_radius=0.9)(x)
    print(x.shape)
    # Combine time series and tabular input
    combined_layer = concatenate([x, tabular_input])
    #Use dense layer from 'myLayer' function of example: 'https://github.com/xiangwenliu/DENN/blob/master/UCI_Data/mylayer.py'
    # x = DENN_Layer(n_units=64, act=gelu,branch=3, name='my_dense_layer_1')(combined_layer)
    # output = DENN_Layer(n_units=1, name='my_dense_layer_3')(x)
    #Dense Layer
    x = Dense(units=64, activation='gelu', name='my_dense_layer_1')(combined_layer)
    output = Dense(units=1, name='my_dense_layer_3')(x)
    model = models.Model(inputs=[ts_input, tabular_input], outputs=output)
    return model
def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

model = build_deepesn_model(ts_steps=seismic_train.shape[1], output_dim=1)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae',rmse])
model.fit(x=[seismic_train, tab_train], y=y_train, batch_size=512,
          epochs=50, validation_data=([seismic_val, tab_val], y_val))

(None, 300)
(None, 300)
(None, 300)
Epoch 1/50
46/46 [==============================] - 8s 161ms/step - loss: 19.3511 - mae: 3.2767 - rmse: 3.9240 - val_loss: 6.3631 - val_mae: 1.5395 - val_rmse: 2.4257
Epoch 2/50
46/46 [==============================] - 7s 151ms/step - loss: 3.6710 - mae: 1.3503 - rmse: 1.8366 - val_loss: 2.5633 - val_mae: 1.2137 - val_rmse: 1.5974
Epoch 3/50
46/46 [==============================] - 7s 149ms/step - loss: 2.5314 - mae: 1.1549 - rmse: 1.5539 - val_loss: 2.1329 - val_mae: 1.0926 - val_rmse: 1.4607
Epoch 4/50
46/46 [==============================] - 7s 152ms/step - loss: 2.1130 - mae: 1.0645 - rmse: 1.4184 - val_loss: 1.7363 - val_mae: 1.0258 - val_rmse: 1.3100
Epoch 5/50
46/46 [==============================] - 7s 153ms/step - loss: 1.7090 - mae: 1.0061 - rmse: 1.2991 - val_loss: 1.5640 - val_mae: 0.9847 - val_rmse: 1.2408
Epoch 6/50
46/46 [==============================] - 7s 152ms/step - loss: 1.4674 - mae: 0.9605 - rmse: 1.2103 - val_loss: 1.4191 - va